<a href="https://colab.research.google.com/github/rubyvanrooyen/data_processing/blob/master/3c39/notebooks/Line_channel_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comet 67P/Churyumov-Gerasimenko line Doppler shift

## Library Installation

Use `skyfield` python package and published data files downloaded from the Minor Planet Center.    
https://rhodesmill.org/skyfield/kepler-orbits.html

In [ ]:
!pip install skyfield

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 442 kB 24.8 MB/s 
     |████████████████████████████████| 46 kB 3.6 MB/s 
     |████████████████████████████████| 228 kB 41.9 MB/s 


Skyfield loads orbital elements from text files using the Pandas library.

In [ ]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Comet target definition
Use Skyfield to map comet ephemeris to sky coordinates given date of interest

In [ ]:
from skyfield.constants import GM_SUN_Pitjeva_2005_km3_s2 as GM_SUN
from skyfield.api import load, Topos
from skyfield.api import S, E, wgs84
from skyfield.data import mpc

In [ ]:
comet_name = '67P/Churyumov-Gerasimenko'

Build a dataframe of comets

In [ ]:
with load.open(mpc.COMET_URL) as f:
    comets = mpc.load_comets_dataframe(f)
# Keep only the most recent orbit for each comet,
# and index by designation for fast lookup.
comets = (comets.sort_values('reference')
          .groupby('designation', as_index=False).last()
          .set_index('designation', drop=False))

print(len(comets), 'comets loaded')
print(mpc.COMET_URL)

[#################################] 100% CometEls.txt


960 comets loaded
https://www.minorplanetcenter.net/iau/MPCORB/CometEls.txt


Read ephemeris of selected comet.    
Cometary orbits are measured centered on the Sun.

You will therefore need to add the barycenter->Sun vector to the Sun->comet vector to produce a position that you can pass to the `observe()` method, which always measures positions from the Solar System barycenter.

In [ ]:
ts = load.timescale()
eph = load('de421.bsp')
sun, earth = eph['sun'], eph['earth']

ephem = comets.loc[comet_name]
print(ephem)
comet = sun + mpc.comet_orbit(ephem, ts, GM_SUN)
print(comet)

[#################################] 100% de421.bsp


designation                            67P/Churyumov-Gerasimenko
perihelion_year                                             2021
perihelion_month                                              11
perihelion_day                                             2.095
perihelion_distance_au                                  1.210563
eccentricity                                            0.649808
argument_of_perihelion_degrees                           22.1462
longitude_of_ascending_node_degrees                      36.3295
inclination_degrees                                       3.8718
magnitude_g                                                 11.0
magnitude_k                                                  4.0
reference                                          MPEC 2022-PB4
Name: 67P/Churyumov-Gerasimenko, dtype: object
Sum of 2 vectors:
 'de421.bsp' segment 0 SOLAR SYSTEM BARYCENTER -> 10 SUN
 _KeplerOrbit 10 SUN -> str


## MeerKAT observer definition

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord, ICRS, AltAz
from astropy.coordinates import Longitude, Latitude, EarthLocation
from astropy.time import Time

Topocentric coordinates specific to MeerKAT telescope's location on the Earth’s surface

In [ ]:
# Apparent topocentric position
meerkat = earth + Topos('30.7130 S', '21.4430 E')
# meerkat = earth + wgs84.latlon(30.7130 * S, 21.4430 * E)

location = EarthLocation.from_geodetic(Longitude('21:26:38.0',
                                                 u.degree,
                                                 wrap_angle=180. * u.degree,
                                                 copy=False),
                                       Latitude('-30:42:39.8',
                                                u.degree,
                                                copy=False),
                                       height=u.Quantity(1086.6,
                                                         u.m,
                                                         copy=False))

## Read time information from observation file

In [ ]:
import numpy as np

```
import numpy as np

msfile='1627186165_sdp_l0-3c39-corr.ms'

# extract the channel frequencies
tb.open(msfile+'/SPECTRAL_WINDOW')
data = tb.getvarcol("NUM_CHAN")
for spw in data.keys():
    n_chans = data[spw][0]
print('Number channels: {}'.format(n_chans))
channelfreqs = tb.getvarcol("CHAN_FREQ")
key = channelfreqs.keys()[0]
print('Number channels: {}'.format(len(channelfreqs[key])))
tb.close()
# write to CSV file
filename = os.path.splitext(msfile)[0]+'_channelfreqs.csv'
with open(filename, 'w') as fout:
    for ch_hz in channelfreqs[key]:
        fout.write('{}\n'.format(ch_hz))

# find unique timestamps indices
tb.open(msfile)
time = tb.getcol("TIME_CENTROID")
len_ = len(time)
uniquetime = list(set(time))
uniquetime = np.sort(uniquetime)
tb.close()
print('{} timestamps with {} unique'.format(len(time), len(uniquetime)))
print('Number dumps: {}'.format(len(uniquetime)))
# write to CSV file
filename = os.path.splitext(msfile)[0]+'_timestamps.csv'
with open(filename, 'w') as fout:
    for ts in uniquetime:
        fout.write('{}\n'.format(ts))
```

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 1627186165_sdp_l0-3c39-corr_timestamps.csv to 1627186165_sdp_l0-3c39-corr_timestamps.csv
Saving 1627186165_sdp_l0-3c39-corr_channelfreqs.csv to 1627186165_sdp_l0-3c39-corr_channelfreqs.csv


In [ ]:
chfile = '1627186165_sdp_l0-3c39-corr_channelfreqs.csv'
with open(chfile, 'r') as fin:
    data = fin.readlines()

channels = []
for chan_ in data:
    channels.append(float(chan_.strip()[1:-1]))
channels = np.array(channels)

In [ ]:
tsfile = '1627186165_sdp_l0-3c39-corr_timestamps.csv'
with open(tsfile, 'r') as fin:
    data = fin.readlines()

epoch = 3506716800.
timestamps = []
for ts_ in data:
    timestamps.append(float(ts_.strip()) - epoch)
timestamps = np.array(timestamps)

## Comet orbit calculation

In [ ]:
from astropy.constants import c
c_kms = c.to('km/s').value
restfreq_hz = 1665.40184e6
# restfreq_hz = 1.66735903e9

In [ ]:
observed_freq_hz = []
for ts_ in timestamps:
    t = ts.from_astropy(Time(ts_, format='unix'))
    # Apparent topocentric position
    apparent = meerkat.at(t).observe(comet).apparent()
    ra, dec, distance = apparent.radec()
    print(f"Topocentric    ({ra.hours}, {dec.degrees}) @ distance {distance.au} [AU] on date {Time(ts_, format='unix').datetime}")
    velocity = apparent.velocity.km_per_s
    vr_kms = np.sqrt(np.sum(velocity**2))
    observed_freq_hz.append(restfreq_hz / (1 + vr_kms/c_kms))

Topocentric    (1.3463497770565924, 3.7051043243151756) @ distance 1.1655918678222172 [AU] on date 2021-07-25 04:20:47.380000
Topocentric    (1.3463534128423502, 3.7051261377607396) @ distance 1.1655904663271668 [AU] on date 2021-07-25 04:20:55.850000
Topocentric    (1.3463570443582846, 3.7051479255381636) @ distance 1.1655890665055242 [AU] on date 2021-07-25 04:21:04.310000
Topocentric    (1.3463606801600396, 3.705169738917046) @ distance 1.1655876650399968 [AU] on date 2021-07-25 04:21:12.780000
Topocentric    (1.3463643159698377, 3.7051915522625567) @ distance 1.1655862635892387 [AU] on date 2021-07-25 04:21:21.250000
Topocentric    (1.3463679518097609, 3.705213365736863) @ distance 1.1655848621542464 [AU] on date 2021-07-25 04:21:29.720000
Topocentric    (1.346371583353101, 3.7052351533224783) @ distance 1.1655834623839323 [AU] on date 2021-07-25 04:21:38.180000
Topocentric    (1.3463752191871936, 3.7052569665679473) @ distance 1.165582060977454 [AU] on date 2021-07-25 04:21:46.650

In [ ]:
def get_channel(obsfreq,channels):
    dnu = 1.633e3
    idx = np.argmin(np.abs(channels - obsfreq))

    if (channels[idx]-observed_freq_hz[0]) < 0:
        ch_end = channels[idx]+dnu/2.
        ch_start = channels[idx+1]-dnu/2.
        # which channel (end>nu or nu>start)
        if ch_end-observed_freq_hz[0] > 0:
            ch_idx = idx
        else:
            ch_idx = idx+1  
    else:
        ch_end = channels[idx-1]+dnu/2.
        ch_start = channels[idx]-dnu/2.
        # which channel (end>nu or nu>start)
        if ch_end-observed_freq_hz[0] > 0:
            ch_idx = idx-1
        else:
            ch_idx = idx
    return ch_idx

In [ ]:
dnu = 1.633e3
line_chans = []
print("ch_idx, chan_start,   freq,         chan_end,     delta.")
for freq in observed_freq_hz:
    ch_idx = get_channel(freq, channels)
    line_chans.append(ch_idx)
    ch_start = channels[ch_idx]-dnu/2.
    ch_end = channels[ch_idx]+dnu/2.
    delta = np.abs(channels[ch_idx]-freq)
    print(f"{ch_idx},  {ch_start/1e9:12.10f}, {freq/1e9:12.10f}, {ch_end/1e9:12.10f}, {delta:3.2f} < {dnu/2.}")

ch_idx, chan_start,   freq,         chan_end,     delta.
15907,  1.6652203935, 1.6652212949, 1.6652220265, 84.90 < 816.5
15907,  1.6652203935, 1.6652212959, 1.6652220265, 85.94 < 816.5
15907,  1.6652203935, 1.6652212970, 1.6652220265, 86.97 < 816.5
15907,  1.6652203935, 1.6652212980, 1.6652220265, 88.00 < 816.5
15907,  1.6652203935, 1.6652212990, 1.6652220265, 89.03 < 816.5
15907,  1.6652203935, 1.6652213001, 1.6652220265, 90.07 < 816.5
15907,  1.6652203935, 1.6652213011, 1.6652220265, 91.10 < 816.5
15907,  1.6652203935, 1.6652213021, 1.6652220265, 92.13 < 816.5
15907,  1.6652203935, 1.6652213032, 1.6652220265, 93.16 < 816.5
15907,  1.6652203935, 1.6652213042, 1.6652220265, 94.19 < 816.5
15907,  1.6652203935, 1.6652213052, 1.6652220265, 95.21 < 816.5
15907,  1.6652203935, 1.6652213062, 1.6652220265, 96.24 < 816.5
15907,  1.6652203935, 1.6652213073, 1.6652220265, 97.27 < 816.5
15907,  1.6652203935, 1.6652213083, 1.6652220265, 98.30 < 816.5
15907,  1.6652203935, 1.6652213093, 1.665222026

In [ ]:
dnu = 1.633e3
idx = np.argmin(np.abs(channels - restfreq_hz))
ch_end = channels[idx-1]+dnu/2.
ch_start = channels[idx]-dnu/2.
# which channel (end>nu or nu>start)
if ch_end-observed_freq_hz[0] > 0:
    ch_idx = idx-1
else:
    ch_idx = idx

In [ ]:
print(f"Observed frequeny expected in channels {np.unique(line_chans)[0]}, obs freq {channels[np.unique(line_chans)][0]/1e9:12.10f} GHz")

Observed frequeny expected in channels 15907, obs freq 1.6652212100 GHz


In [ ]:
print(f"Rest frequeny expected in channels {ch_idx}, rest freq {restfreq_hz/1e9:12.10f} GHz")

Rest frequeny expected in channels 16017, rest freq 1.6654018400 GHz


In [ ]:
print(f"Extract line region ch 14787 = {channels[14787]/1e9:12.10f} GHz to ch 16628 = {channels[16628]/1e9:12.10f} GHz")
print(f"Extract line region ch 16385 = {channels[16385]/1e9:12.10f} GHz to ch 18227 = {channels[18227]/1e9:12.10f} GHz")

Extract line region ch 14787 = 1.6633925900 GHz to ch 16628 = 1.6663983800 GHz
Extract line region ch 16385 = 1.6660016300 GHz to ch 18227 = 1.6690090500 GHz


In [ ]:
from astropy.constants import c
c_kms = c.to('km/s').value

import astropy.units as u
restfreq = 1.66735903*u.GHz

radio_equiv = u.doppler_radio(restfreq)
# restfreq_hz = 1665.40184e6

observed_freq = 1.6671788000*u.GHz

radio_velocity = observed_freq.to(u.km/u.s, equivalencies=radio_equiv)
print(radio_velocity)


32.40549499728321 km / s
